In [246]:
import pandas as pd
import numpy as np
from acquire_prepare import get_data_sql, prepare, parse_log_entry, summarize

In [48]:
df = get_data_sql()

In [49]:
# drops nulls
df = df.drop('deleted_at', axis=1).dropna()

In [50]:
# creates series of list from path columns
split_path = df.path.str.split('/')

In [51]:
# drops 'content' from each list
for ls in split_path:
    if ls[0] == 'content':
        ls.remove('content')

# drops 'further-reading' from each list
for ls in split_path:
    if len(ls) > 1 and ls[1] == 'further-reading':
        ls.remove('further-reading')

In [52]:
# parses the remaining lists in dplit path and puts them in the originfal dataframe as new columns
df[['topic', 'lesson']] = split_path.apply(parse_log_entry)

In [54]:
# converts the date and time to correct dtype and overwrites old date column
df['date'] = pd.to_datetime((df['date'] +' '+ df['time']))

# drops the unneeded time col
df.drop(['time', 'id'], axis=1, inplace=True)

# change cohort id to an integer
df['cohort_id'] = df.cohort_id.astype(int)

In [56]:
# how can I filter the df so that I only get the dates and times for when they were active
# convert the date and times for strart and end date to datetime
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

In [55]:
df.head()

,date,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson
0,2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,,
1,2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,
2,2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,object-oriented-programming
3,2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,slides,object_oriented_programming
4,2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,javascript-i,conditionals


### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
> #### Plan:
> * create separate df for each program (make it digestible)
> * check the amount of cohorts in each
> * groupby by cohorts
> * count how many time a lesson has been clicked on here
> * for each cohort grouping sort descending
> * get the head for each

In [5]:
# checking data
df.head()

,date,path,user_id,cohort_id,ip,name,program_id,topic,lesson
0,2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,1,,
1,2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,1,java-ii,
2,2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,1,java-ii,object-oriented-programming
3,2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,1,slides,object_oriented_programming
4,2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,Teddy,2,javascript-i,conditionals


In [280]:
# creating different df for each program id
# what are the differnt program_ids
df.program_id.value_counts()

# separating
program_one = df[df['program_id'] == 1]
program_two = df[df['program_id'] == 2]
program_three = df[df['program_id'] == 3]
program_four = df[df['program_id'] == 4]

In [281]:
# program one
pd.DataFrame(program_one.groupby('cohort_id')['lesson'].value_counts())[program_one.groupby('cohort_id')['lesson'].value_counts()>900]\
.rename({'lesson':'lesson_count'}, axis=1)\
.sort_values('lesson_count', ascending=False)

,,lesson_count
cohort_id,lesson,
14,,2780
1,,1831
17,,958


In [65]:
# program one
# pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['topic'].value_counts())[program_one.groupby('cohort_id')['topic'].value_counts()>900]\
# .rename({'topic':'topic_count'}, axis=1)\
# .sort_values('topic_count', ascending=False)

In [282]:

# program one
pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
[program_one.groupby('cohort_id')['topic'].value_counts()>900]\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(1)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/563413257.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/563413257.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\


,,0
cohort_id,topic,
17,javascript-i,953
14,spring,1104
1,javascript-i,1333


In [283]:
# program two
pd.DataFrame(program_two.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
[program_two.groupby('cohort_id')['topic'].value_counts()>1000]\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(1)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/532094218.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_two.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/532094218.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.DataFrame(program_two.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\


,,0
cohort_id,topic,
139,javascript-i,1331
138,javascript-i,2902
135,javascript-i,5849
134,javascript-i,3644
132,javascript-i,4049
62,javascript-i,5364
61,javascript-i,2904
58,javascript-i,5147
57,javascript-i,4852


In [57]:
# program three
pd.DataFrame(program_three.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
[program_three.groupby('cohort_id')['topic'].value_counts()>1000]\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(2)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1391/3559874815.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_three.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1391/3559874815.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.DataFrame(program_three.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\


0
cohort_id topic               
137       fundamentals    3451
          sql             2277
133       classification  2715
          fundamentals    2469
59        classification  5030
                          2980
55        3-sql           2499
          4-python        2330
34        1-fundamentals  3124
          6-regression    2573

In [64]:
# program four
pd.DataFrame(program_four.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(2)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1391/722415719.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_four.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\


0
cohort_id topic      
9         html-css  4
                    1

#### Takeaways
* Program 1: javascript-i
* Program 2: javasvript-i
* Program 3: fundamentals, closely followed by classification
* Program 4: html-css

### 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
> #### Plan:
> * What is significance in this case? 
> * For each lesson, we can calculate an overall average of for each cohort. For each overall average, we can identify if there is

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
> #### Plan:
> * First, we need to find out what 'when active' means. Each cohort must be filtered for only their active dates. 
> * After this, I can answer this question by grouping by the user ids. With each user id, I can take a count of how many times they occur in the dataframe. After taking the count, I can get summary statistics about each, using the iqr to identify when 

In [45]:
# checking col names
df.head()

,date,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson
0,2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,/,
1,2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,j,a
2,2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,j,a
3,2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,s,l
4,2018-01-26 09:56:24,javascript-i/conditionals,2,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,j,a


In [50]:
# start date and end date values
(df.start_date.value_counts() == df.start_date.value_counts()).value_counts()

True    44
Name: start_date, dtype: int64

In [53]:
# there are 47 cohorts and 44 start and end dates
len(df.cohort_id.value_counts())

47

In [62]:
# next, i will focus on one cohort
df.groupby('cohort_id')['user_id'].nunique().idxmax()

28

something is wrong with cohort 28
* they have too many user and all the start and end dats are the same

In [57]:
# set index to date
play = df.copy()
play.set_index('date', inplace=True)

In [58]:
# nothing showing up?
mask = (play.cohort_id == 2)
play[mask][(play[mask].index == play[mask]['end_date'])]

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson
date,,,,,,,,,,,,,


In [59]:
# creating new df to hold activity during active times in the program
active_df = pd.DataFrame(columns=play.columns)

In [60]:
# checking that its empty
active_df

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson


In [61]:
# creating mask and setup for getting the df
mask = (play.cohort_id == 31)
addition = play[mask][(play[mask].index >= play[mask]['start_date']) & (play[mask].index <= play[mask]['end_date'])]

In [64]:
# mergind the data
active_df = pd.concat([active_df, addition])

In [66]:
# active_df

In [74]:
# new clean df
active_df = pd.DataFrame(columns=play.columns)

# For each cohort in cohort_id
for i in play.cohort_id.unique():
    
    # mask it with the cohort
    mask = (play.cohort_id == i)

    # get all values for the active user during this time
    filtered_addition = play[mask][(play[mask].index >= play[mask]['start_date']) & (play[mask].index <= play[mask]['end_date'])]
    
    # concatinating the data onto the blank df for each
    active_df = pd.concat([active_df, filtered_addition])

In [76]:
# this is a df with all activity from users while they were activiely in the program
# now we can analyze this info
active_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 647197 entries, 2018-01-26 09:56:24 to 2021-04-21 13:14:10
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   path        647197 non-null  object        
 1   user_id     647197 non-null  object        
 2   cohort_id   647197 non-null  object        
 3   ip          647197 non-null  object        
 4   name        647197 non-null  object        
 5   slack       647197 non-null  object        
 6   start_date  647197 non-null  datetime64[ns]
 7   end_date    647197 non-null  datetime64[ns]
 8   created_at  647197 non-null  object        
 9   updated_at  647197 non-null  object        
 10  program_id  647197 non-null  object        
 11  topic       647197 non-null  object        
 12  lesson      647197 non-null  object        
dtypes: datetime64[ns](2), object(11)
memory usage: 69.1+ MB


In [77]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [211]:
# getting the values and frequencies 
user_df = value_counts_and_frequencies(active_df['user_id'])

In [212]:
# check
user_df.head()

,count,proba
344,4763,0.007359
570,4075,0.006296
685,3223,0.004980
671,3193,0.004934
555,3086,0.004768


In [213]:
# getting users that barely accessed the data
user_df['count'].nsmallest(n = 50, keep='all')

879     1
619     1
918     1
940     1
832     3
278     4
539     5
956     6
812     7
388     8
679    11
572    12
941    12
697    13
889    16
968    17
487    17
963    18
340    18
961    20
49     25
64     25
772    26
24     26
41     28
976    29
972    34
973    35
529    36
959    41
967    41
981    42
371    42
971    44
964    45
741    47
960    48
56     52
543    53
46     54
978    55
965    55
962    58
57     62
858    63
45     65
36     65
975    67
250    67
969    71
Name: count, dtype: int64

In [113]:
# getting the count of the bottom 10 percent
under_ten_perc = user_df['count'].quantile(.10)
# getting the user_id's with less than 10 percent of clicks
users_under_ten_ls = list(user_df[user_df['count'] < under_ten_perc].index)

In [123]:
# some cohorts have many user_ids with low amounts, I'm guesing that some are people who did not stay in the program while others may be something of concer
active_df[active_df['user_id'].isin(users_under_ten_ls)]\
.groupby('cohort_id')['user_id'].nunique()

cohort_id
21     12
22      2
23      1
24      4
27      1
29      2
31      3
33      1
34      1
52      3
53      1
57      2
58      1
59      2
61      2
62      3
132     1
133     1
135     2
138     6
139    22
Name: user_id, dtype: int64

In [144]:
# get the last click per user and compare it to the start and end date
final_use_10_perc = active_df[active_df['user_id'].isin(users_under_ten_ls)]\
.sort_index()\
.groupby('user_id').tail(1)

In [152]:
final_use_10_perc['end_date_diff'] = final_use_10_perc.index - final_use_10_perc['end_date']

In [153]:
final_use_10_perc['start_date_diff'] = final_use_10_perc.index - final_use_10_perc['start_date']

In [154]:
# majority
summarize(final_use_10_perc)

                    SUMMARY REPORT


Dataframe head: 


,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson,end_date_diff,start_date_diff
2018-02-05 09:08:24,spring/fundamentals/form-model-binding,49,21,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,2,spring,fundamentals,-10 days +09:08:24,131 days 09:08:24
2018-02-05 10:40:44,spring/extra-features/json-response,45,21,104.193.225.250,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,2,spring,extra-features,-10 days +10:40:44,131 days 10:40:44
2018-02-05 15:37:36,mysql/tables,55,21,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,2017-09-27 20:22:41,2017-09-27 20:22:41,2,mysql,tables,-10 days +15:37:36,131 days 15:37:36




Dataframe info: 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 73 entries, 2018-02-05 09:08:24 to 2021-04-21 12:32:01
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   path             73 non-null     object         
 1   user_id          73 non-null     object         
 2   cohort_id        73 non-null     object         
 3   ip               73 non-null     object         
 4   name             73 non-null     object         
 5   slack            73 non-null     object         
 6   start_date       73 non-null     datetime64[ns] 
 7   end_date         73 non-null     datetime64[ns] 
 8   created_at       73 non-null     object         
 9   updated_at       73 non-null     object         
 10  program_id       73 non-null     object         
 11  topic            73 non-null     object         
 12  lesson           73 non-null     object         
 13  end_date_diff    73 non-n

""




Dataframe Description: 


...

,start_date_diff
"(0 days 10:11:59.249999999, 17 days 03:19:52]",41
"(17 days 03:19:52, 33 days 16:29:27]",0
"(33 days 16:29:27, 50 days 05:39:02]",5
"(50 days 05:39:02, 66 days 18:48:37]",0
"(66 days 18:48:37, 83 days 07:58:12]",3
"(83 days 07:58:12, 99 days 21:07:47]",1
"(99 days 21:07:47, 116 days 10:17:22]",1
"(116 days 10:17:22, 132 days 23:26:57]",8
"(132 days 23:26:57, 149 days 12:36:32]",13
"(149 days 12:36:32, 166 days 01:46:07]",1


nulls in dataframe by column: 


,num_rows_missing,percent_rows_missing
path,0,0.0
user_id,0,0.0
cohort_id,0,0.0
ip,0,0.0
name,0,0.0
slack,0,0.0
start_date,0,0.0
end_date,0,0.0
created_at,0,0.0
updated_at,0,0.0


nulls in dataframe by row: 


,num_cols_missing,percent_cols_missing
2018-02-05 09:08:24,0,0.0
2020-09-23 14:54:20,0,0.0
2021-04-20 16:28:32,0,0.0
2021-04-20 15:25:42,0,0.0
2021-04-20 12:04:53,0,0.0
...,...,...
2019-07-29 12:42:51,0,0.0
2019-07-22 11:13:31,0,0.0
2019-03-29 13:36:04,0,0.0
2019-03-19 12:19:23,0,0.0


#### Takeaways
* Majority of the last uses end at day 17 (41 of them)
    * This may be students who decided not to continue with the program
* Oberon (139) and Sequoia (21) have an abnormally high number of users (22, 12) that use it extremely little and discontinue use later on 
* Vast majority of the users are in web development programs, with 4 in a datascience cohort

In [87]:
def get_lower_and_upper_bounds(df, col, k=1.5):
    '''
    Takes in a df and a column names and return the lower fence and upper fence with a k of 1.5
    '''
    # first quartile
    q1 = df[col].quantile(.25)
    
    # second quartile
    q3 = df[col].quantile(.75)
    
    # calculate the iqr
    iqr = (q3 - q1)
    
    # get the upper fence
    upper_fence = q3 + (k * iqr)
    
    # get the lower fence
    lower_fence = q1 - (k * iqr)
    
    # exit and return the upper and lower fence
    return lower_fence, upper_fence

In [91]:
get_lower_and_upper_bounds(user_df, 'count')\
, get_lower_and_upper_bounds(user_df, 'proba')

((-874.5, 2501.5), (-0.0013512114549356692, 0.003865129164690195))

In [100]:
x = user_df['count']
# Calculate the z-score 
zscores = pd.Series((x - x.mean()) / x.std())

# Finds all of the observations two standard deviations or more.
x[zscores <= -1.2]

# Finds all of the observations three standard deviations or more
x[zscores.abs() >= 3]

344    4763
570    4075
685    3223
671    3193
555    3086
764    2955
423    2946
823    2946
760    2923
794    2852
Name: count, dtype: int64

In [101]:
user_df.describe()

,count,proba
count,731.000000,731.000000
mean,885.358413,0.001368
std,646.913125,0.001000
min,1.000000,0.000002
25%,391.500000,0.000605
50%,795.000000,0.001228
75%,1235.500000,0.001909
max,4763.000000,0.007359


### 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
> #### Plan:
> * What defines suspicious activity?
> * What does webscraping look like in tabular data?
> * What consititutes suspicious ip addresses?

Yes, there is quite a bit of suspicious activity happening
There are also a lot of suspivious ip addresses that come up a lot

In [179]:
def acquire(file_name, column_names):
    return pd.read_csv(file_name, sep="\s", header=None, names=column_names, usecols=[0, 2, 3, 4, 5])

def prep(df, user):
    df = df[df.user_id == user]
    pages = df['path'].resample('d').count()
    return pages

def compute_pct_b(pages, span, weight, user):
    midband = pages.ewm(span=span).mean()
    stdev = pages.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    bb = pd.concat([ub, lb], axis=1)
    my_df = pd.concat([pages, midband, bb], axis=1)
    my_df.columns = ['pages', 'midband', 'ub', 'lb']
    my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plt_bands(my_df, user):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

def find_anomalies(df, user, span, weight):
    pages = prep(df, user)
    my_df = compute_pct_b(pages, span, weight, user)
    # plt_bands(my_df, user)
    return my_df[my_df.pct_b>1]


In [216]:
user = 1
span = 30
weight = 6
user_anomaly_df = find_anomalies(df, user, span, weight)

anomalies = pd.DataFrame()
user_anomaly_df = find_anomalies(df, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)

In [218]:
span = 30
weight = 3.5

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_anomaly_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_anomaly_df], axis=0)

In [219]:
page_anomaly_ld = list(anomalies.pages.value_counts().nsmallest(keep='all').index)

In [220]:
user_anomalies_ls = list(anomalies[anomalies.pages.isin(page_anomaly_ld)]['user_id'].values)

In [221]:
df[df['user_id'].isin(user_anomalies_ls)].groupby('user_id').count()

,path,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson,date
user_id,,,,,,,,,,,,,
33,151,151,151,151,151,151,151,151,151,151,151,151,151
72,530,530,530,530,530,530,530,530,530,530,530,530,530
101,224,224,224,224,224,224,224,224,224,224,224,224,224
138,1264,1264,1264,1264,1264,1264,1264,1264,1264,1264,1264,1264,1264
153,760,760,760,760,760,760,760,760,760,760,760,760,760
223,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316,1316
248,5075,5075,5075,5075,5075,5075,5075,5075,5075,5075,5075,5075,5075
255,114,114,114,114,114,114,114,114,114,114,114,114,114
264,2083,2083,2083,2083,2083,2083,2083,2083,2083,2083,2083,2083,2083


In [236]:
df[df['user_id'].isin(user_anomalies_ls)].ip.value_counts(normalize=True)

97.105.19.58       0.322147
70.112.179.142     0.066937
173.173.115.201    0.042256
72.190.187.173     0.030371
99.88.62.179       0.029184
                     ...   
64.134.155.129     0.000016
174.128.180.24     0.000016
107.77.217.49      0.000016
24.173.208.242     0.000016
107.77.221.36      0.000016
Name: ip, Length: 318, dtype: float64

In [238]:
suspicious_ls = user_df[user_df.index.isin(user_anomalies_ls)].index

In [242]:
df[df['user_id'].isin(suspicious_ls)].ip.value_counts()

97.105.19.58      17583
70.112.179.142     4117
72.190.187.173     1868
99.88.62.179       1795
75.1.166.163       1769
                  ...  
99.203.213.177        1
107.77.220.123        1
12.5.63.210           1
107.77.221.36         1
174.207.25.208        1
Name: ip, Length: 278, dtype: int64

In [233]:
# anomalies.user_id user_df.index

### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
> #### Plan:
> * Identify which program is ds and which is wd, add column called program_type
> * Identify which paths are ds and which are wd, create a column called path_type
> * Identify when the change in ability happen
> * Check to see if there have been anytimes that program_type != path_type

In [264]:
df['in_webdev'] = np.where((df['program_id'] == 1), 1,\
         np.where((df['program_id'] == 2), 1,\
                  np.where((df['program_id'] == 4), 1, 0)))

array([1, 1, 1, ..., 1, 1, 1])

In [274]:
# yes, this is happening
df[['in_webdev', 'path']]

,in_webdev,path
date,,
2018-01-26 09:55:03,1,/
2018-01-26 09:56:02,1,java-ii
2018-01-26 09:56:05,1,java-ii/object-oriented-programming
2018-01-26 09:56:06,1,slides/object_oriented_programming
2018-01-26 09:56:24,1,javascript-i/conditionals
...,...,...
2021-04-21 16:41:51,1,jquery/personal-site
2021-04-21 16:42:02,1,jquery/mapbox-api
2021-04-21 16:42:09,1,jquery/ajax/weather-map


### 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
> #### Plan:
> * Create a df with only post-graduation curriculum accesses
> * Get value_counts from those times

In [301]:
# new clean df
inactive_df = pd.DataFrame(columns=play.columns)

# For each cohort in cohort_id
for i in df.cohort_id.unique():
    
    # mask it with the cohort
    mask = (play.cohort_id == i)

    # get all values for the active user during this time
    filtered_addition = play[mask][play[mask].index > play[mask]['end_date']]
    
    # concatinating the data onto the blank df for each
    inactive_df = pd.concat([inactive_df, filtered_addition])

In [302]:
inactive_df.head()

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson
2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,,
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,object-oriented-programming
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,slides,object_oriented_programming
2018-01-26 10:40:15,javascript-i/functions,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,javascript-i,functions


In [303]:
# separating
program_one = inactive_df[inactive_df['program_id'] == 1]
program_two = inactive_df[inactive_df['program_id'] == 2]
program_three = inactive_df[inactive_df['program_id'] == 3]
program_four = inactive_df[inactive_df['program_id'] == 4]

In [305]:
# checking
program_one.head()

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson
2018-01-26 09:55:03,/,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,,
2018-01-26 09:56:02,java-ii,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,java-ii,object-oriented-programming
2018-01-26 09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,slides,object_oriented_programming
2018-01-26 10:40:15,javascript-i/functions,1,8,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,javascript-i,functions


In [324]:
# program 
pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
[program_one.groupby('cohort_id')['topic'].value_counts()>100]\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(2)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/1388460011.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/1388460011.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.DataFrame(program_one.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\


0
cohort_id topic             
19        laravel        187
          mysql          182
17        javascript-i   953
          html-css       636
14        spring        1104
          javascript-i  1006
13        laravel        591
          php_iv         273
8         java-iii       212
                         210
7         html-css       165
          javascript-i   150
1         javascript-i  1333
          html-css      1063

In [307]:
program_two.head()

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id,topic,lesson
2018-05-17 10:30:24,/,18,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,,
2018-05-17 10:30:27,mysql,18,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,mysql,
2018-05-17 10:30:41,mysql/basic-statements,18,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,mysql,basic-statements
2018-05-17 11:20:04,/,6,22,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,,
2018-05-17 13:57:07,jquery,6,22,64.132.225.203,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,jquery,


In [308]:
# program two
pd.DataFrame(program_two.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
[program_two.groupby('cohort_id')['topic'].value_counts()>100]\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(1)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/134903616.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_two.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\
/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/134903616.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.DataFrame(program_two.groupby(['cohort_id', 'topic'])['cohort_id', 'topic'].value_counts())\


,,0
cohort_id,topic,
61,spring,332
58,spring,252
57,javascript-i,594
56,java-ii,702
53,spring,464
52,javascript-i,323
51,javascript-i,1407
33,javascript-i,809
32,javascript-i,555


In [325]:
# program 4
pd.DataFrame(program_three.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\
[program_three.groupby('cohort_id')['path'].value_counts()>100]\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(3)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/3560508571.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_three.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\
/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/3560508571.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pd.DataFrame(program_three.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\


0
cohort_id path                                         
59        /                                         386
          search/search_index.json                  149
          sql/mysql-overview                        123
55        /                                         569
          search/search_index.json                  157
          classification/overview                   136
34        /                                         491
          search/search_index.json                  187
          1-fundamentals/modern-data-scientist.jpg  116

In [316]:
# program three
pd.DataFrame(program_four.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').head(3)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/1965904642.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_four.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\


0
cohort_id path                                          
9         content/html-css                             2
          /                                            1
          content/html-css/gitbook/images/favicon.ico  1

### 7. Which lessons are least accessed?
> #### Plan:
> * 

In [323]:
# program one
pd.DataFrame(program_one.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').tail(1)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/1301382914.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_one.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\


,,0
cohort_id,path,
19,spring/setup,1
17,spring/extra-features/json-response,1
14,spring/fundamentals/integration-tests,1
13,spring/setup,1
12,prework/cli/02-listing-files,1
11,search/search_index.json,1
8,uploads/58a217a705bde.jpg,1
7,spring/fundamentals/controllers,1
6,spring/fundamentals/controllers,1


In [322]:
# program two
pd.DataFrame(program_two.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').tail(1)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/2376498038.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_two.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\


,,0
cohort_id,path,
62,web-design/ux/purpose,1
61,slides/syntax_types_and_variables,1
58,web-design/ux/purpose,1
57,web-design/ux/purpose,1
56,web-design/ux/purpose,1
53,web-design/ux/purpose,1
52,web-design/ui/visuals,1
51,syntax-types-and-variables,1
33,web-design/ux/purpose,1


In [327]:
# program three
pd.DataFrame(program_three.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\
.sort_values(['cohort_id', 0], ascending=False)\
.groupby('cohort_id').tail(5)

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1889/336019201.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pd.DataFrame(program_three.groupby(['cohort_id', 'path'])['cohort_id', 'path'].value_counts())\


0
cohort_id path                                    
59        git                                    1
          individual-project/individual-project  1
          python/type-annotations                1
          regression/explore                     1
          stats/sampling                         1
55        storytelling/present                   1
          storytelling/refine                    1
          storytelling/tableau                   1
          storytelling/understand                1
          timeseries/svr                         1
34        storytelling/refine                    1
          storytelling/tableau                   1
          tidy-data                              1
          timeseries/svr                         1
          viz.py                                 1

### 8. Anything else I should be aware of?
> #### Plan:
> * 